# **Klasifikasi Pengenalan Audio**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PSD/audio/emotional audio

Mounted at /content/drive
/content/drive/MyDrive/PSD/audio/emotional audio


In [ ]:
%%capture
pip install librosa

# **DESKRIPSI DATA**

Dataset ini untuk melatih klasifikasi emosi (7 emosi utama) dalam audio. Ada satu set 200 kata target yang diucapkan dalam frasa pembawa "Say the word _' oleh dua aktris (berusia 26 dan 64 tahun) dan rekaman dibuat dari set tersebut yang menggambarkan masing-masing tujuh emosi (marah, jijik, takut, bahagia, kejutan menyenangkan, kesedihan, dan netral). Total ada 2800 data (file audio).

Dataset tersebut disusun sedemikian rupa sehingga masing-masing dari dua aktor wanita dan emosi mereka terkandung dalam foldernya sendiri. Dan di dalamnya, semua file audio 200 kata target dapat ditemukan. Format file audio adalah format WAV

data didapatkan dari https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess

### **Penjelasan Label**

*'OAF_angry': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi marah dari pembicara OAF.

*'OAF_disgust': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi jijik atau muak dari pembicara OAF.

*'OAF_Fear': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi takut dari pembicara OAF.

*'OAF_happy': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi bahagia dari pembicara OAF.

*'OAF_neutral': mencerminkan ekspresi wajah atau suara yang terkait dengan ekspresi wajah atau suara netral dari pembicara OAF.

*'OAF_Pleasant_surprise': mencerminkan ekspresi wajah atau suara yang terkait dengan kejutan menyenangkan dari pembicara OAF.

*'OAF_Sad': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi sedih dari pembicara OAF.

Untuk Pembicara YAF:

*'YAF_angry': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi marah dari pembicara YAF.

*'YAF_disgust': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi jijik atau muak dari pembicara YAF.

*'YAF_fear': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi takut dari pembicara YAF.

*'YAF_happy': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi bahagia dari pembicara YAF.

*'YAF_neutral': mencerminkan ekspresi wajah atau suara yang terkait dengan ekspresi wajah atau suara netral dari pembicara YAF.

*'YAF_Pleasant_surprise': mencerminkan ekspresi wajah atau suara yang terkait dengan kejutan menyenangkan dari pembicara YAF.

*'YAF_sad': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi sedih dari pembicara YAF.




# **EKSTRAKSI FITUR**

In [ ]:
# Import Library

import os
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode

In [ ]:
# Daftar folder yang akan dianalisis
folders = [
    'OAF_angry','OAF_disgust','OAF_Fear','OAF_happy','OAF_neutral','OAF_Pleasant_surprise','OAF_Sad','YAF_angry','YAF_disgust','YAF_fear','YAF_happy','YAF_neutral','YAF_pleasant_surprised','YAF_sad'
]

In [ ]:
def calculate_statistics(audio_path):
    y, sr = librosa.load(audio_path)

    # UNTUK MENGHITUNG NILAI ZCR
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])

    # UNTUK MENGHITUNG NILAI RMSE
    rmse = np.sum(y**2) / len(y)
    rmse_median = np.median(y**2)
    rmse_std_dev = np.std(y**2)
    rmse_kurtosis = kurtosis(y**2)
    rmse_skew = skew(y**2)

    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew, rmse, rmse_median, rmse_std_dev, rmse_kurtosis, rmse_skew]

**Rumus-rumus dari hasil Ekstraksi Fitur Audio**

1.  Mean (Rata-rata)

  Rata-rata adalah nilai tengah dari suatu set data. Ini dihitung dengan menjumlahkan semua nilai dalam dataset dan kemudian membaginya dengan jumlah total data.

  $\text{{Mean}} = \frac{1}{n} \sum_{i=1}^{n} y_i$

2.  Standard Deviation (Deviasi Standar)

  Deviasi standar mengukur seberapa tersebar atau jauh nilai-nilai individu dalam dataset dari nilai rata-ratanya.

  $\text{{Standard Deviation}} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2}$

3. Median

  Median adalah nilai tengah dari suatu set data yang telah diurutkan. Jika jumlah elemen dalam dataset ganjil, median adalah nilai di tengah. Jika genap, median adalah rata-rata dari dua nilai tengah.

  $\text{{Median}} = \begin{cases}
      y_{(\frac{n}{2})} & \text{{if }} n \text{{ is odd}} \\
      \frac{1}{2} (y_{(\frac{n}{2})} + y_{(\frac{n}{2} + 1)}) & \text{{if }} n \text{{ is even}}
   \end{cases}$

4. Skewness

  Skewness mengukur sejauh mana distribusi data miring dari simetri. Nilai skewness positif menunjukkan ekor distribusi yang lebih panjang di sebelah kanan, sedangkan nilai negatif menunjukkan ekor distribusi yang lebih panjang di sebelah kiri.

  $\text{{Skewness}} = \frac{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^3}{\left(\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2\right)^{3/2}}$

5. Kurtosis

  Kurtosis mengukur tingkat kecuraman (tumpukan atau landasan) distribusi data. Kurtosis tinggi menunjukkan distribusi yang lebih fokus di sekitar rata-rata, sementara kurtosis rendah menunjukkan distribusi yang lebih datar. Kurtosis dibandingkan dengan kurtosis distribusi normal standar yang memiliki nilai 3.

  $\text{{Kurtosis}} = \frac{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^4}{\left(\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2\right)^2} - 3$


Fungsi 'calculate_statistics' digunakan untuk menghitung beberapa statistik dari sinyal audio yang diberikan

1. Zero Crossing Rate (ZCR):

  Definisi: Nilai ini mengukur seberapa sering sinyal melintasi nilai nol per detik.

  **Rumus:**
  
  * zcr_mean: Nilai rata-rata ZCR.
  * zcr_median: Nilai median ZCR.
  * zcr_std_dev: Deviasi standar dari ZCR.
  * zcr_kurtosis: Kurtosis dari distribusi ZCR.
  * zcr_skew: Skewness dari distribusi ZCR.
  
2. Root Mean Square Error (RMSE):

  Definisi: RMSE mengukur rata-rata perbedaan antara nilai aktual dan nilai yang diprediksi (dalam hal ini, amplitudo sinyal audio).

  **Rumus:**

  * rmse: Nilai RMSE dari sinyal.
  * rmse_median: Nilai median dari RMSE.
  * rmse_std_dev: Deviasi standar dari RMSE.
  * rmse_kurtosis: Kurtosis dari distribusi RMSE.
  * rmse_skew: Skewness dari distribusi RMSE.

In [ ]:
# Membuat DataFrame untuk menyimpan hasil
data = []

In [ ]:
# Loop melalui setiap folder dan file audio
for folder in folders:
    folder_path = f'{folder}'
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            statistics = calculate_statistics(audio_path)
            data.append([folder, filename] + statistics)

In [ ]:
# Membuat DataFrame dari data
columns =  ['Label', 'File'] + ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']
df = pd.DataFrame(data, columns=columns)

In [ ]:
# Menyimpan DataFrame ke dalam file CSV
df.to_csv('hasil_audio.csv', index=False)

In [ ]:
# Menampilkan file CSV
df

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,OAF_angry,OAF_bar_angry.wav,0.093492,0.073486,0.092156,5.375023,2.351386,0.000816,0.000065,0.002035,38.330481,5.086085
1,OAF_angry,OAF_bath_angry.wav,0.107017,0.055176,0.122474,3.534486,2.090227,0.003370,0.000184,0.008584,28.569717,4.586325
2,OAF_angry,OAF_back_angry.wav,0.085165,0.057129,0.096892,6.195356,2.569610,0.003138,0.000161,0.008551,37.627046,5.372503
3,OAF_angry,OAF_base_angry.wav,0.135888,0.071289,0.133905,1.334860,1.560679,0.001682,0.000076,0.004909,38.826234,5.510271
4,OAF_angry,OAF_cab_angry.wav,0.092378,0.065430,0.088781,5.642932,2.346017,0.000746,0.000068,0.001859,32.876072,4.855687
...,...,...,...,...,...,...,...,...,...,...,...,...
2817,YAF_sad,YAF_young_sad.wav,0.118967,0.038574,0.194587,4.191204,2.367201,0.001229,0.000189,0.002673,25.149270,4.197964
2818,YAF_sad,YAF_witch_sad.wav,0.168584,0.039307,0.227741,1.044324,1.531838,0.001462,0.000142,0.003381,26.135000,4.261510
2819,YAF_sad,YAF_which_sad.wav,0.157758,0.038086,0.222642,1.170585,1.582929,0.003143,0.000411,0.006965,22.366196,4.141749
2820,YAF_sad,YAF_yes_sad.wav,0.172475,0.035889,0.248237,0.546175,1.487487,0.002709,0.000381,0.005774,16.285086,3.668968


# **PREPROCESSING MENGGUNAKAN Z-SCORE**

**Z-score** (Z-skor) adalah metode normalisasi yang mengubah data ke dalam distribusi normal standar dengan rata-rata (mean) 0 dan deviasi standar (standard deviation) 1.

**StandardScaler** adalah alat yang digunakan untuk menerapkan metode Z-score ini ke data Anda. Dengan kata lain, StandardScaler adalah implementasi konkret dari normalisasi Z-score dalam scikit-learn.

Rumus:

$z = \frac{{x - \mu}}{{\sigma}}$


Penjelasan:

*   Z adalah Z-score
*   x adalah nilai individu
*   μ adalah rata-rata dari data set
*   σ adalah deviasi standar dari data set

Z-score menyediakan cara untuk mengukur seberapa ekstrim atau umum suatu nilai dalam distribusi data. Interpretasi Z-score sebagai berikut:

- Jika Z-score positif, titik data berada di atas rata-rata populasi.
- Jika Z-score negatif, titik data berada di bawah rata-rata populasi.
- Nilai Z-score nol menunjukkan bahwa titik data sama dengan rata-rata populasi.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [ ]:
# Baca data dari file CSV
dataknn= pd.read_csv('hasil_audio.csv')
# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label','File'], axis=1)  # Ganti 'target_column' dengan nama kolom target
y = dataknn['Label']
# split data into train and test sets
X_train,X_test,y_train, y_test= train_test_split(X, y, random_state=1, test_size=0.2)
# define scaler
scaler = StandardScaler()
# fit scaler on the training dataset
scaler.fit(X_train)
# save the scaler
dump(scaler, open('scaler.pkl', 'wb'))
# transform the training dataset
# normalisasi data train
X_train_scaled = scaler.transform(X_train)
dataknn

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,OAF_angry,OAF_bar_angry.wav,0.093492,0.073486,0.092156,5.375023,2.351386,0.000816,0.000065,0.002035,38.330481,5.086085
1,OAF_angry,OAF_bath_angry.wav,0.107017,0.055176,0.122474,3.534486,2.090227,0.003370,0.000184,0.008584,28.569717,4.586325
2,OAF_angry,OAF_back_angry.wav,0.085165,0.057129,0.096892,6.195356,2.569610,0.003138,0.000161,0.008551,37.627046,5.372503
3,OAF_angry,OAF_base_angry.wav,0.135888,0.071289,0.133905,1.334860,1.560679,0.001682,0.000076,0.004909,38.826234,5.510271
4,OAF_angry,OAF_cab_angry.wav,0.092378,0.065430,0.088781,5.642932,2.346017,0.000746,0.000068,0.001859,32.876072,4.855687
...,...,...,...,...,...,...,...,...,...,...,...,...
2817,YAF_sad,YAF_young_sad.wav,0.118967,0.038574,0.194587,4.191204,2.367201,0.001229,0.000189,0.002673,25.149270,4.197964
2818,YAF_sad,YAF_witch_sad.wav,0.168584,0.039307,0.227741,1.044324,1.531838,0.001462,0.000142,0.003381,26.135000,4.261510
2819,YAF_sad,YAF_which_sad.wav,0.157758,0.038086,0.222642,1.170585,1.582929,0.003143,0.000411,0.006965,22.366196,4.141749
2820,YAF_sad,YAF_yes_sad.wav,0.172475,0.035889,0.248237,0.546175,1.487487,0.002709,0.000381,0.005774,16.285086,3.668968


In [ ]:
import pickle
with open('scaler.pkl', 'rb') as standarisasi:
    loadscal= pickle.load(standarisasi)

In [ ]:
#normalisasi X testing dari hasil normalisasi X train yang disimpan dalam model
X_test_scaled=loadscal.transform(X_test)

# **METODE KNN**

**K-Nearest Neighbors (KNN)** adalah algoritma pembelajaran mesin yang digunakan untuk tugas klasifikasi dan regresi. Algoritma ini merupakan salah satu metode pembelajaran mesin yang termasuk dalam kategori pembelajaran berbasis instan (instance-based learning) atau pembelajaran berbasis data.

Klasifikasi dengan KNN:
Dalam konteks klasifikasi, KNN bekerja dengan cara sebagai berikut:

* Penentuan Tetangga Terdekat: Mengukur jarak antara data baru (poin uji) dengan semua titik data latih.perhitungan jarak antara data baru (poin uji) dengan titik-titik data latih merupakan komponen penting dari algoritma K-Nearest Neighbors (KNN). Biasanya, metrik jarak yang digunakan dalam KNN adalah Euclidean distance, meskipun metrik lain seperti Manhattan distance atau Minkowski distance juga bisa digunakan tergantung pada kebutuhan. Berikut merupakan penjelasan terkait jarak Euclidean antara dua vektor (\(x\) dan \(y\)) dalam ruang berdimensi \(n\) dihitung sebagai akar kuadrat dari jumlah kuadrat perbedaan antara setiap elemen:

Euclidean Distance = $\sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}$

* Pemilihan K Tetangga Terdekat: Memilih K tetangga terdekat dari data baru berdasarkan jarak.
* Voting atau Weighted Voting: Menentukan label atau nilai target untuk data baru dengan melihat mayoritas kelas dari K tetangga terdekat. Dalam beberapa implementasi, bobot dapat diterapkan pada voting berdasarkan jarak.

**Rumus:**

$\hat{y} = \text{majority}(\{y_i \,|\, (x_i, y_i) \in \text{k-nearest neighbors of } x\})$

Penjelasan:

- $\hat{y}$ adalah prediksi kelas untuk data baru.
- $\text{majority}$ mengacu pada proses pemilihan mayoritas kelas dari K tetangga terdekat.
- $\{y_i | (x_i, y_i) \in \text{k-nearest neighbors of } x\}$ adalah himpunan dari label kelas dari K tetangga terdekat dari data baru (x).
- K tetangga terdekat dipilih berdasarkan jarak dari data baru.

Rumus ini memperlihatkan bahwa dalam KNN, prediksi kelas untuk data baru diambil dari mayoritas kelas dari K tetangga terdekatnya. Jika kelas mayoritas dari tetangga terdekat adalah kelas 'A', maka data baru akan diprediksi sebagai kelas 'A'.


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
K = 50
acc = np.zeros((K-1))

for n in range(1,K,2):
    knn = KNeighborsClassifier(n_neighbors= n, metric = "euclidean").fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)

    acc[n-1]= accuracy_score(y_test,y_pred)

print('akurasi terbaik adalah ', acc.max(), 'dengan nilai k =', acc.argmax()+1)

akurasi terbaik adalah  0.679646017699115 dengan nilai k = 7


In [ ]:
knn = KNeighborsClassifier(n_neighbors= 7, metric = "euclidean")
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=7)

In [ ]:
y_pred = knn.predict(X_test_scaled)
y_pred

array(['YAF_fear', 'YAF_pleasant_surprised', 'OAF_happy', 'YAF_neutral',
       'YAF_fear', 'YAF_sad', 'YAF_neutral', 'YAF_neutral', 'YAF_happy',
       'OAF_happy', 'OAF_angry', 'OAF_angry', 'OAF_neutral',
       'YAF_pleasant_surprised', 'YAF_pleasant_surprised', 'YAF_fear',
       'YAF_sad', 'YAF_disgust', 'OAF_Pleasant_surprise', 'YAF_sad',
       'OAF_Sad', 'YAF_angry', 'OAF_disgust', 'YAF_pleasant_surprised',
       'OAF_disgust', 'YAF_happy', 'OAF_happy', 'YAF_angry',
       'OAF_neutral', 'OAF_angry', 'OAF_Pleasant_surprise', 'OAF_Fear',
       'OAF_disgust', 'OAF_neutral', 'YAF_neutral', 'YAF_happy',
       'YAF_neutral', 'OAF_Pleasant_surprise', 'YAF_neutral', 'YAF_fear',
       'OAF_happy', 'YAF_sad', 'YAF_neutral', 'YAF_happy', 'YAF_disgust',
       'OAF_angry', 'OAF_disgust', 'YAF_neutral', 'YAF_angry', 'YAF_fear',
       'YAF_disgust', 'YAF_angry', 'YAF_happy', 'YAF_angry', 'YAF_fear',
       'OAF_Pleasant_surprise', 'YAF_disgust', 'OAF_Fear', 'YAF_fear',
       'YAF_fear

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
print("Akurasi:",accuracy)

Akurasi: 0.679646017699115


In [ ]:
dump(knn, open('modelknn.pkl', 'wb'))

# **REDUKSI DATA DENGAN PCA**

Reduksi ada 2 jenis bisa menggunakan seleksi data atau transformation data (contoh nya PCA / Principal component anlysis).
seleksi data kita dapat emmilih fitur berdasarkan yang fitur/ kolom yang paling berpengaruh.
sedangkan transformasi data kita perlu membuat kooordinat baru dari dari fitur yang ada sejumlah dengan jumlah fitur nya.
mengapa reduksi data atau data reduction di perlukan?
karna terlalu banyak kolom/fitur/ciri yang harus dikenali tidak baik untuk pemprosesan data dan memakan waktu komputasi yang lama, maka dari itu reduksi data di lakukan untuk mendapatkan data yang terbaik. Rumus PCA sebagai berikut:

1. **Hitung Matriks Kovarian:**

   $ \text{Cov}(\mathbf{X}) = \frac{1}{n-1} \sum_{i=1}^{n} (\mathbf{X}_i - \bar{\mathbf{X}})(\mathbf{X}_i - \bar{\mathbf{X}})^T $

    di mana:


  *   $\mathbf{X}_i \text{ adalah vektor data poin ke-i,}$
  *   $\bar{\mathbf{X}} \text{ adalah vektor rata-rata dari semua data,}$
  *   $n \text{ adalah jumlah data poin.} $

2. **Hitung Nilai dan Vektor Eigen:** Menghitung nilai eigen (λ) dan vektor eigen (v) dari matriks kovarian.

   $ \text{Cov}(\mathbf{X})\mathbf{v} = \lambda\mathbf{v} $

3. **Pilih Principal Components:** Principal components dipilih berdasarkan nilai eigen tertinggi. Misalnya, jika kita ingin mereduksi dimensi menjadi k, kita memilih k vektor eigen dengan nilai eigen tertinggi.

4. **Transformasi Data:** Data asli (X) dapat diproyeksikan ke dalam ruang yang baru (Y) yang terdiri dari principal components yang dipilih.

   $ \mathbf{Y} = \mathbf{X} \mathbf{V} $

    di mana:

*   (Y) : matriks yang berisi data yang telah diproyeksikan ke dalam ruang principal components,
*   (X) : matriks data asli,
*   (V) : matriks yang berisi vektor eigen sebagai kolom-kolomnya.

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=8)
X_train_pca= sklearn_pca.fit_transform(X_train_scaled)
type(X_train_pca)

numpy.ndarray

pca = PCA(n_components=8): yaitu membuat objek PCA dengan menyertakan
parameter `n_components=8`. `n_components` adalah jumlah komponen utama (principal components) yang ingin dipertahankan setelah reduksi dimensi. Dalam hal ini akan digunakan 8 komponen utama.


pca_train = pca.fit_transform(X_train_scaled): dimana menggunakan objek PCA yang telah dibuat (`pca`) untuk melakukan reduksi dimensi pada data latih (`X_train_scaled`).`fit_transform` digunakan untuk menghitung komponen utama dan mengubah data latih ke dalam ruang dimensi yang baru.Kemudian Hasilnya disimpan dalam variabel `pca_train`.

data latih (`X_train_scaled`) sudah dinormalisasi sebelumnya (dengan menggunakan z-score normalization), dan PCA digunakan untuk menghasilkan representasi data yang lebih ringkas dengan hanya menggunakan 8 komponen utama.

In [ ]:
X_train_pca.shape #untuk mengetahui tentang jumlah baris dan kolom dari data hasil PCA pada data uji.

(2257, 8)

In [ ]:
X_test_pca.shape #untuk mengetahui tentang jumlah baris dan kolom dari data hasil transformasi PCA pada data latih.

(565, 8)

In [ ]:
dump(sklearn_pca, open('PCA8.pkl', 'wb')) #simpan pca

In [ ]:
import pickle
with open('PCA8.pkl', 'rb') as pca:
  loadpca= pickle.load(pca)

In [ ]:
pca_test = loadpca.transform(X_test_scaled) #untuk mentransformasi data uji menjadi ruang fitur baru yang dibentuk oleh komponen-komponen PCA
pca_test.shape

(565, 8)

# **PEMODELAN KKN MENGGUNAKAN PCA**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=7)
classifier.fit(X_train_pca, y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_prediksi = classifier.predict(X_test_pca)
y_prediksi

array(['YAF_fear', 'YAF_pleasant_surprised', 'OAF_happy', 'YAF_neutral',
       'YAF_fear', 'YAF_sad', 'YAF_neutral', 'YAF_neutral', 'YAF_happy',
       'OAF_happy', 'OAF_angry', 'OAF_angry', 'OAF_neutral',
       'YAF_pleasant_surprised', 'YAF_pleasant_surprised', 'YAF_fear',
       'YAF_sad', 'YAF_disgust', 'OAF_Pleasant_surprise', 'YAF_sad',
       'OAF_Sad', 'YAF_angry', 'OAF_disgust', 'YAF_pleasant_surprised',
       'OAF_disgust', 'YAF_happy', 'OAF_happy', 'YAF_angry',
       'OAF_neutral', 'OAF_angry', 'OAF_Pleasant_surprise', 'OAF_Fear',
       'OAF_disgust', 'OAF_neutral', 'YAF_neutral', 'YAF_happy',
       'YAF_neutral', 'OAF_Pleasant_surprise', 'YAF_neutral', 'YAF_fear',
       'OAF_happy', 'YAF_sad', 'YAF_neutral', 'YAF_happy', 'YAF_disgust',
       'OAF_angry', 'OAF_disgust', 'YAF_neutral', 'YAF_angry', 'YAF_fear',
       'YAF_disgust', 'YAF_angry', 'YAF_happy', 'YAF_angry', 'YAF_fear',
       'OAF_Pleasant_surprise', 'YAF_disgust', 'OAF_Fear', 'YAF_fear',
       'YAF_fear

In [ ]:
acc_pca= accuracy_score(y_test,y_prediksi)
print("Akurasi:",acc_pca)

Akurasi: 0.6743362831858407


# **LINK STREAMLIT**

link streamlit klasifikasi audio ada di bawah ini :

https://uts-klasifikasi-suara.streamlit.app/